In [1]:
#from __future__ import (absolute_import, division, print_function,
#                        unicode_literals)

import datetime
import os.path
import sys

import backtrader as bt


In [2]:
"""
A long only strategy: buy if price has been falling 3 sessions in a row; exit after 5 sessions.
"""
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 5),
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()}, {txt}")
    
    def log_order(self, order: bt.order.Order):
        direction = 'BUY' if order.isbuy() else 'SELL'
        self.log(f"{direction} EXECUTED, Size: {order.executed.size}, Price: {order.executed.price:.2f}, Cost: {order.executed.value}, Comm: {order.executed.comm:.2f}")

    def __init__(self):
        self.dataclose = self.datas[0].close

        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order: bt.order.Order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log_order(order)
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log_order(order)
            
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")
        
        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade: bt.trade.Trade):
        if not trade.isclosed:
            return
        self.log(f"OPERATION PROFIT, Gross {trade.pnl:.2f}, Net {trade.pnlcomm:.2f}")

    def next(self):
        self.log(f'Close Price: {self.dataclose[0]:.2f}')

        # if there is a pending order, we will not send a 2nd one
        if self.order:
            return
    
        # check if we are in the market
        if not self.position:
            # BUY if the price has been falling 3 sessions in a row
            if self.dataclose[0] < self.dataclose[-1]:
                if self.dataclose[-1] < self.dataclose[-2]:
                    self.log(f"BUY CREATE, {self.dataclose[0]:.2f}")
                    # keep track of the created order to avoid a 2nd order
                    self.order = self.buy()
        else:
            if len(self) >= (self.bar_executed + self.params.exitbars):
                self.log(f"SELL CREATE, {self.dataclose[0]:.2f}")
                # keep track of the created order to avoid a 2nd order
                self.order = self.sell()


In [3]:
"""
A long only strategy: buy if close is greate than SMA; sell if the close is smaller than the SMA.
"""
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()}, {txt}")
    
    def log_order(self, order: bt.order.Order):
        direction = 'BUY' if order.isbuy() else 'SELL'
        self.log(f"{direction} EXECUTED, Size: {order.executed.size}, Price: {order.executed.price:.2f}, Cost: {order.executed.value}, Comm: {order.executed.comm:.2f}")

    def __init__(self):
        self.dataclose = self.datas[0].close

        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)

    def notify_order(self, order: bt.order.Order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log_order(order)
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log_order(order)
            
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")
        
        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade: bt.trade.Trade):
        if not trade.isclosed:
            return
        self.log(f"OPERATION PROFIT, Gross {trade.pnl:.2f}, Net {trade.pnlcomm:.2f}")

    def next(self):
        self.log(f'Close Price: {self.dataclose[0]:.2f}')

        # if there is a pending order, we will not send a 2nd one
        if self.order:
            return
    
        # check if we are in the market
        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.log(f"BUY CREATE, {self.dataclose[0]:.2f}")
                self.order = self.buy()
        
        else:
            if self.dataclose[0] < self.sma[0]:
                self.log(f"SELL CREATE, {self.dataclose[0]:.2f}")
                self.order = self.sell()


In [4]:
cerebro = bt.Cerebro()

cerebro.addstrategy(TestStrategy)

datapath = 'data/orcl-1995-2014.txt'
data = bt.feeds.YahooFinanceCSVData(
    dataname=datapath,
    fromdate=datetime.datetime(2000, 1, 1),
    todate=datetime.datetime(2000, 12, 31),
    reverse=False)
cerebro.adddata(data)

cerebro.broker.setcash(1000)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.addsizer(bt.sizers.FixedSize, stake=10)

cerebro.broker.setcommission(commission=0.0)

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 1000.00
2000-01-24, Close Price: 24.10
2000-01-25, Close Price: 25.10
2000-01-25, BUY CREATE, 25.10
2000-01-26, BUY EXECUTED, Size: 10, Price: 25.24, Cost: 252.39999999999998, Comm: 0.00
2000-01-26, Close Price: 24.49
2000-01-27, Close Price: 23.04
2000-01-27, SELL CREATE, 23.04
2000-01-28, SELL EXECUTED, Size: -10, Price: 22.90, Cost: 252.39999999999998, Comm: 0.00
2000-01-28, OPERATION PROFIT, Gross -23.40, Net -23.40
2000-01-28, Close Price: 21.07
2000-01-31, Close Price: 22.22
2000-02-01, Close Price: 24.02
2000-02-02, Close Price: 24.16
2000-02-02, BUY CREATE, 24.16
2000-02-03, BUY EXECUTED, Size: 10, Price: 24.63, Cost: 246.29999999999998, Comm: 0.00
2000-02-03, Close Price: 25.21
2000-02-04, Close Price: 25.71
2000-02-07, Close Price: 26.66
2000-02-08, Close Price: 26.49
2000-02-09, Close Price: 26.66
2000-02-10, Close Price: 27.71
2000-02-11, Close Price: 26.55
2000-02-14, Close Price: 27.66
2000-02-15, Close Price: 27.30
2000-02-16, Close Price: 27.24

In [31]:
from backtrader_plotly.plotter import BacktraderPlotly
from backtrader_plotly.scheme import PlotScheme
from plotly.io import renderers

renderers.default = "browser"

PLOT_SCHEME = PlotScheme(decimal_places=5, max_legend_text_width=16)

figs = cerebro.plot(BacktraderPlotly(show=True, scheme=PLOT_SCHEME))